<font size="4">This jupyter notebook is intended to scrap flight deals from the website **Kayak** and turn them into a csv. There is also a functionality for sending emails with the new prices.</font>


*You need to <font color='red'>install</font> package <b><a href="https://pypi.org/project/selenium/">selenium</a></b> and <font color='red'>download</font> <b><a href="https://sites.google.com/chromium.org/driver/downloads?authuser=0">ChromeDriver</a></b>*

*Credits to:*
***Fábio Neves***

*Link to his article:*
***https://medium.com/@fneves/if-you-like-to-travel-let-python-help-you-scrape-the-best-fares-5a1f26213086***

*Link to his YouTube explanation:*
***https://www.youtube.com/watch?v=nN0OD6HLDJk***
_____

In [66]:
from time import sleep, strftime
from random import randint
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import smtplib
from email.mime.multipart import MIMEMultipart

In [67]:
chromedriver_path = 'C:/Users/Sotiris/Downloads/chromedriver_win32/chromedriver.exe' # Change this to your own chromedriver path!


In [68]:
driver = webdriver.Chrome(executable_path=chromedriver_path)
sleep(2)

In [69]:
kayak = 'https://www.kayak.com/flights/DUS-SKG/2021-12-23-flexible/2022-01-07-flexible?sort=bestflight_a'
driver.get(kayak)
sleep(3)

In [70]:
# Closing the popup

# xp_popup_close = '//button[contains(@id,"dialog-close") and contains(@class,"Button-No-Standard-Style close ")]'
# driver.find_elements_by_xpath(xp_popup_close)[5].click()

In [82]:
# Load more results to maximize the scraping

def load_more():
    try:
        more_results = '//a[@class = "moreButton"]'
        driver.find_element_by_xpath(more_results).click()
        print('sleeping.....')
        sleep(randint(10,15))
    except:
        pass

In [90]:
def start_kayak(city_from, city_to, date_start, date_end):
    """City codes - it's the IATA codes!
    Date format -  YYYY-MM-DD"""
    
    kayak = ('https://www.kayak.com/flights/' + city_from + '-' + city_to +
             '/' + date_start + '-flexible/' + date_end + '-flexible?sort=bestflight_a')
    driver.get(kayak)
    sleep(randint(10,15))
    
    # sometimes a popup shows up, so we can use a try statement to check it and close
    try:
        xp_popup_close = '//button[contains(@id,"dialog-close") and contains(@class,"Button-No-Standard-Style close ")]'
        driver.find_elements_by_xpath(xp_popup_close)[5].click()
    except Exception as e:
        pass
    sleep(randint(10,15))
    print('loading more.....')
    
#     load_more()
    
    print('starting first scrape.....')
    df_flights_best = page_scrape()
    df_flights_best['sort'] = 'best'
    sleep(randint(10,15))
    
    # Let's also get the lowest prices from the matrix on top
    matrix = driver.find_elements_by_xpath('//*[contains(@id,"FlexMatrixCell")]')
    matrix_prices = [price.text.replace('$','') for price in matrix]
    matrix_prices = list(map(int, matrix_prices))
    matrix_min = min(matrix_prices)
    matrix_avg = sum(matrix_prices)/len(matrix_prices)
    
    print('switching to cheapest results.....')
    cheap_results = '//a[@data-code = "price"]'
    driver.find_element_by_xpath(cheap_results).click()
    sleep(randint(10,15))
    print('loading more.....')
    
#     load_more()
    
    print('starting second scrape.....')
    df_flights_cheap = page_scrape()
    df_flights_cheap['sort'] = 'cheap'
    sleep(randint(10,15))
    
    print('switching to quickest results.....')
    quick_results = '//a[@data-code = "duration"]'
    driver.find_element_by_xpath(quick_results).click()  
    sleep(randint(10,15))
    print('loading more.....')
    
#     load_more()
    
    print('starting third scrape.....')
    df_flights_fast = page_scrape()
    df_flights_fast['sort'] = 'fast'
    sleep(randint(8,10))
    
    # saving a new dataframe as an excel file. the name is custom made to your cities and dates
    final_df = df_flights_cheap.append(df_flights_best).append(df_flights_fast)
    final_df.to_excel('{}_flights_{}-{}_from_{}_to_{}.xlsx'.format(strftime("%Y%m%d-%H%M"),
                                                                                   city_from, city_to, 
                                                                                   date_start, date_end), index=False)
    print('saved df.....')
    
    # We can keep track of what they predict and how it actually turns out!
#     xp_loading = '//div[contains(@id,"advice")]'
#     loading = driver.find_element_by_xpath(xp_loading).text
#     xp_prediction = '//span[@class="info-text"]'
#     prediction = driver.find_element_by_xpath(xp_prediction).text
#     print(loading+'\n'+prediction)
    
#     # sometimes we get this string in the loading variable, which will conflict with the email we send later
#     # just change it to "Not Sure" if it happens
#     weird = '¯\\_(ツ)_/¯'
#     if loading == weird:
#         loading = 'Not sure'
    
#     username = 'YOUREMAIL@hotmail.com'
#     password = 'YOUR PASSWORD'

#     server = smtplib.SMTP('smtp.outlook.com', 587)
#     server.ehlo()
#     server.starttls()
#     server.login(username, password)
#     msg = ('Subject: Flight Scraper\n\n\
# Cheapest Flight: {}\nAverage Price: {}\n\nRecommendation: {}\n\nEnd of message'.format(matrix_min, matrix_avg, (loading+'\n'+prediction)))
#     message = MIMEMultipart()
#     message['From'] = 'YOUREMAIL@hotmail.com'
#     message['to'] = 'YOUROTHEREMAIL@domain.com'
#     server.sendmail('YOUREMAIL@hotmail.com', 'YOUROTHEREMAIL@domain.com', msg)
#     print('sent email.....')

In [73]:
def page_scrape():
    """This function takes care of the scraping part"""
    
    xp_sections = '//*[@class="section duration allow-multi-modal-icons"]'
    sections = driver.find_elements_by_xpath(xp_sections)
    sections_list = [value.text for value in sections]
    section_a_list = sections_list[::2] # This is to separate the two flights
    section_b_list = sections_list[1::2] # This is to separate the two flights
    
    # if you run into a reCaptcha, you might want to do something about it
    # you will know there's a problem if the lists above are empty
    # this if statement lets you exit the bot or do something else
    # you can add a sleep here, to let you solve the captcha and continue scraping
    # i'm using a SystemExit because i want to test everything from the start
    if section_a_list == []:
        raise SystemExit
    
    # I'll use the letter A for the outbound flight and B for the inbound
    a_duration = []
    a_section_names = []
    for n in section_a_list:
        # Separate the time from the cities
        a_section_names.append(''.join(n.split()[2:5]))
        a_duration.append(''.join(n.split()[0:2]))
    b_duration = []
    b_section_names = []
    for n in section_b_list:
        # Separate the time from the cities
        b_section_names.append(''.join(n.split()[2:5]))
        b_duration.append(''.join(n.split()[0:2]))

    xp_dates = '//div[@class="section date"]'
    dates = driver.find_elements_by_xpath(xp_dates)
    dates_list = [value.text for value in dates]
    a_date_list = dates_list[::2]
    b_date_list = dates_list[1::2]
    # Separating the weekday from the day
    a_day = [value.split()[0] for value in a_date_list]
    a_weekday = [value.split()[1] for value in a_date_list]
    b_day = [value.split()[0] for value in b_date_list]
    b_weekday = [value.split()[1] for value in b_date_list]
    
    # getting the prices
    xp_prices = '//a[@class="booking-link "]/span[@class="price option-text"]'
    prices = driver.find_elements_by_xpath(xp_prices)
    prices_list = [price.text.replace('$','') for price in prices if price.text != '']
    prices_list = list(map(int, prices_list))
    
    # the stops are a big list with one leg on the even index and second leg on odd index
    xp_stops = '//div[@class="section stops"]/div[1]'
    stops = driver.find_elements_by_xpath(xp_stops)
    stops_list = [stop.text[0].replace('n','0') for stop in stops]
    a_stop_list = stops_list[::2]
    b_stop_list = stops_list[1::2]

    xp_stops_cities = '//div[@class="section stops"]/div[2]'
    stops_cities = driver.find_elements_by_xpath(xp_stops_cities)
    stops_cities_list = [stop.text for stop in stops_cities]
    a_stop_name_list = stops_cities_list[::2]
    b_stop_name_list = stops_cities_list[1::2]
    
    # this part gets me the airline company and the departure and arrival times, for both legs
    xp_schedule = '//div[@class="section times"]'
    schedules = driver.find_elements_by_xpath(xp_schedule)
    hours_list = []
    carrier_list = []
    for schedule in schedules:
        hours_list.append(schedule.text.split('\n')[0])
        carrier_list.append(schedule.text.split('\n')[1])
    # split the hours and carriers, between a and b legs
    a_hours = hours_list[::2]
    a_carrier = carrier_list[::2]
    b_hours = hours_list[1::2]
    b_carrier = carrier_list[1::2]

    
    cols = (['Out Day', 'Out Time', 'Out Weekday', 'Out Airline', 'Out Cities', 'Out Duration', 'Out Stops', 'Out Stop Cities',
            'Return Day', 'Return Time', 'Return Weekday', 'Return Airline', 'Return Cities', 'Return Duration', 'Return Stops', 'Return Stop Cities',
            'Price'])
    
    flights_df = pd.DataFrame({'Out Day': a_day,
                               'Out Weekday': a_weekday,
                               'Out Duration': a_duration,
                               'Out Cities': a_section_names,
                               'Return Day': b_day,
                               'Return Weekday': b_weekday,
                               'Return Duration': b_duration,
                               'Return Cities': b_section_names,
                               'Out Stops': a_stop_list,
                               'Out Stop Cities': a_stop_name_list,
                               'Return Stops': b_stop_list,
                               'Return Stop Cities': b_stop_name_list,
                               'Out Time': a_hours,
                               'Out Airline': a_carrier,
                               'Return Time': b_hours,
                               'Return Airline': b_carrier,                           
                               'Price': prices_list})[cols]
    
    flights_df['timestamp'] = strftime("%Y%m%d-%H%M") # so we can know when it was scraped
    return flights_df

In [91]:
city_from = input('From which city? ')
city_to = input('Where to? ')
date_start = input('Search around which departure date? Please use YYYY-MM-DD format only ')
date_end = input('Return when? Please use YYYY-MM-DD format only ')

# city_from = 'LIS'
# city_to = 'SIN'
# date_start = '2019-08-21'
# date_end = '2019-09-07'

for n in range(0,1):
    start_kayak(city_from, city_to, date_start, date_end)
    print('iteration {} was complete @ {}'.format(n, strftime("%Y%m%d-%H%M")))
    
    # Wait 4 hours
#     sleep(60*60*4)
#     print('sleep finished.....')


From which city? DUS
Where to? SKG
Search around which departure date? Please use YYYY-MM-DD format only 2021-12-23
Return when? Please use YYYY-MM-DD format only 2022-01-07
loading more.....
starting first scrape.....
switching to cheapest results.....
loading more.....
starting second scrape.....
switching to quickest results.....
loading more.....
starting third scrape.....
saved df.....
iteration 0 was complete @ 20210814-0659


In [55]:
test_df = page_scrape()


eft
[<selenium.webdriver.remote.webelement.WebElement (session="3f38e3ad88e8d7710928a558e6e0399d", element="81818786-d936-4e2e-b053-24de83ada7bb")>, <selenium.webdriver.remote.webelement.WebElement (session="3f38e3ad88e8d7710928a558e6e0399d", element="f4bfed97-66fa-4202-a282-3da66cb56d3f")>, <selenium.webdriver.remote.webelement.WebElement (session="3f38e3ad88e8d7710928a558e6e0399d", element="383af85d-51bd-4583-be7b-73d3cec786f0")>, <selenium.webdriver.remote.webelement.WebElement (session="3f38e3ad88e8d7710928a558e6e0399d", element="eb35b6c9-767c-4ab3-80e9-8b528c191dc8")>, <selenium.webdriver.remote.webelement.WebElement (session="3f38e3ad88e8d7710928a558e6e0399d", element="5b540fee-0fde-49cb-83b6-622e27f8c251")>, <selenium.webdriver.remote.webelement.WebElement (session="3f38e3ad88e8d7710928a558e6e0399d", element="e782cec0-8d48-44c0-a2ac-b2f92f2e1a2a")>, <selenium.webdriver.remote.webelement.WebElement (session="3f38e3ad88e8d7710928a558e6e0399d", element="60fb9ac9-5139-47b6-b84a-2800

In [56]:
test_df

,Out Day,Out Time,Out Weekday,Out Airline,Out Cities,Out Duration,Out Stops,Out Stop Cities,Return Day,Return Time,Return Weekday,Return Airline,Return Cities,Return Duration,Return Stops,Return Stop Cities,Price,timestamp
0,12/22,8:45 am – 1:50 pm,Wed,Lufthansa,DUS‐SKG,4h05m,1,FRA,1/7,2:35 pm – 10:20 pm,Fri,Lufthansa,SKG‐DUS,8h45m,1,FRA,186,20210814-0625
1,12/21,9:25 am – 2:20 pm,Tue,Lufthansa,DUS‐SKG,3h55m,1,MUC,1/10,2:35 pm – 5:50 pm,Mon,Lufthansa,SKG‐DUS,4h15m,1,FRA,168,20210814-0625
2,12/23,12:50 pm – 4:40 pm,Thu,Aegean Airlines,DUS‐SKG,2h50m,0,,1/4,10:10 am – 12:00 pm,Tue,Aegean Airlines,SKG‐DUS,2h50m,0,,230,20210814-0625
3,12/21,12:50 pm – 4:40 pm,Tue,Aegean Airlines,DUS‐SKG,2h50m,0,,1/9,10:10 am – 12:00 pm,Sun,Aegean Airlines,SKG‐DUS,2h50m,0,,230,20210814-0625
4,12/24,3:40 pm – 7:25 pm,Fri,Ryanair,NRN‐SKG,2h45m,0,,1/7,7:50 pm – 9:45 pm,Fri,Ryanair,SKG‐NRN,2h55m,0,,251,20210814-0625
5,12/24,3:40 pm – 7:25 pm,Fri,Ryanair,NRN‐SKG,2h45m,0,,1/7,10:10 am – 12:00 pm,Fri,Aegean Airlines,SKG‐DUS,2h50m,0,,263,20210814-0625
6,12/24,3:40 pm – 7:25 pm,Fri,Ryanair,NRN‐SKG,2h45m,0,,1/8,10:10 am – 12:00 pm,Sat,Aegean Airlines,SKG‐DUS,2h50m,0,,265,20210814-0625
7,12/24,3:40 pm – 7:25 pm,Fri,Ryanair,NRN‐SKG,2h45m,0,,1/6,10:10 am – 12:00 pm,Thu,Aegean Airlines,SKG‐DUS,2h50m,0,,271,20210814-0625
8,12/21,12:50 pm – 4:40 pm,Tue,Aegean Airlines,DUS‐SKG,2h50m,0,,1/10,10:10 am – 12:00 pm,Mon,Aegean Airlines,SKG‐DUS,2h50m,0,,283,20210814-0625
9,12/25,12:50 pm – 4:40 pm,Sat,Aegean Airlines,DUS‐SKG,2h50m,0,,1/6,10:10 am – 12:00 pm,Thu,Aegean Airlines,SKG‐DUS,2h50m,0,,283,20210814-0625


In [10]:
# Bonus: save a screenshot!
driver.save_screenshot('pythonscraping.png')

True

In [62]:
driver.quit()